# Section 1: Imports and Configuration

In [0]:
import pandas as pd
import sys
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp, to_json, from_json, col, udf, explode, lit
from pyspark.sql.functions import col, concat_ws
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType, TimestampType, DoubleType, BooleanType
from pyspark.sql.functions import to_timestamp, col
import requests
from datetime import datetime
import json
import os
import re
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

# Configuration
CATALOG_NAME = "tulip_sandbox"  # Change this to your catalog name
SCHEMA_NAME = "tulip_tables"         # Change this to your schema name
METADATA_SCHEMA_NAME = "tulip_tables_metadata" # Separate schema for metadata
BATCH_SIZE = 100
MAX_RETRIES = 3

# Section 2: API Configuration and Client

In [0]:
@dataclass
class TulipConfig:
    auth_header: str
    base_url: str
    
    @classmethod
    def from_env(cls) -> 'TulipConfig':
        auth_header = "Basic " + os.getenv('AUTH')
        base_url = 'https://william.tulip.co/api/v3'
        return cls(auth_header=auth_header, base_url=base_url)

class TulipAPI:
    def __init__(self, config: TulipConfig):
        self.config = config
        self.headers = {
            'Authorization': config.auth_header,
            'Content-Type': 'application/json'
        }
    
    def get_tables(self) -> List[Dict[str, Any]]:
        response = requests.get(f"{self.config.base_url}/tables", headers=self.headers)
        response.raise_for_status()
        return response.json()
    
    def get_table_records(self, table_id: str, offset: int = 0) -> List[Dict[str, Any]]:
        params = {
            "limit": str(BATCH_SIZE),
            "offset": str(offset),
            "includeTotalCount": "true",
            "filterAggregator": "all"
        }
        response = requests.get(
            f"{self.config.base_url}/tables/{table_id}/records",
            headers=self.headers,
            params=params
        )
        response.raise_for_status()
        return response.json()

# Section 3: Utility Functions

In [0]:
def sanitize_table_name(name: str) -> str:
    return re.sub(r'[^a-zA-Z0-9_]', '_', name).lower()

def map_tulip_type_to_spark_type(tulip_type: str) -> Any:
    """Map Tulip data types to Spark data types objects"""
    type_mapping = {
        'string': StringType(),
        'float': DoubleType(),
        'boolean': BooleanType(),
        'integer': IntegerType(),
        'timestamp': TimestampType(),
        'interval': StringType(),
        'color': StringType(),
        'imageUrl': StringType(),
        'tableLink': StringType()
    }
    return type_mapping.get(tulip_type, StringType())  # Default to StringType for unknown types

def validate_schema_compatibility(spark: SparkSession, df, table_name: str) -> bool:
    """Validate if DataFrame schema is compatible with existing table"""
    try:
        # Get the table schema
        table_df = spark.table(table_name)
        table_schema = table_df.schema
        df_schema = df.schema
        
        # Check for columns that exist in both with different types
        for table_field in table_schema.fields:
            for df_field in df_schema.fields:
                if table_field.name == df_field.name and table_field.dataType != df_field.dataType:
                    print(f"Schema conflict: Column '{table_field.name}' has different types - "
                          f"Table: {table_field.dataType}, DataFrame: {df_field.dataType}")
                    return False
        
        return True
    except Exception as e:
        print(f"Error validating schema: {str(e)}")
        return True  # Assume compatible if we can't check

# Section 4: Unity Catalog Table Management

In [0]:
def create_metadata_tables(spark: SparkSession):
    # Create metadata schema if it doesn't exist
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{METADATA_SCHEMA_NAME}")
    
    # Create tables table
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{METADATA_SCHEMA_NAME}.tables (
            id STRING,
            name STRING,
            description STRING,
            workspace_id STRING,
            created_at TIMESTAMP,
            updated_at TIMESTAMP
        ) USING DELTA
    """)
    
    # Create columns table
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{METADATA_SCHEMA_NAME}.columns (
            table_id STRING,
            column_name STRING,
            column_label STRING,
            data_type STRING,
            is_unique BOOLEAN
        ) USING DELTA
    """)

def update_metadata_tables(spark: SparkSession, table_metadata: List[Dict[str, Any]]):
    # Create tables DataFrame
    tables_data = [
        (table["id"], sanitize_table_name(table["label"]), table["description"], 
         table["workspaceID"], table["createdAt"], table["updatedAt"])
        for table in table_metadata
    ]
    tables_df = spark.createDataFrame(
        tables_data,
        ["id", "name", "description", "workspace_id", "created_at", "updated_at"]
    )
    
    # Create columns DataFrame
    columns_data = []
    for table in table_metadata:
        for column in table["columns"]:
            if not column["hidden"]:
                columns_data.append((
                    table["id"],
                    column["name"],
                    column["label"],
                    map_tulip_type_to_spark_type(column["dataType"]["type"]).typeName(),
                    column["unique"]
                ))
    columns_df = spark.createDataFrame(
        columns_data,
        ["table_id", "column_name", "column_label", "data_type", "is_unique"]
    )
    
    # Update tables
    tables_df.createOrReplaceTempView("tables_df_view")
    spark.sql(f"""
        MERGE INTO {CATALOG_NAME}.{METADATA_SCHEMA_NAME}.tables AS target
        USING tables_df_view AS source
        ON target.id = source.id
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)
    
    # Update columns
    columns_df.createOrReplaceTempView("columns_df_view")
    spark.sql(f"""
        MERGE INTO {CATALOG_NAME}.{METADATA_SCHEMA_NAME}.columns AS target
        USING columns_df_view AS source
        ON target.table_id = source.table_id AND target.column_name = source.column_name
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

# Section 5: Data Table Management

In [0]:
def build_table_schema(table_metadata: Dict[str, Any]) -> StructType:
    """Build Spark schema from table metadata with robust type handling"""
    fields = []
    for col in table_metadata['columns']:
        if not col['hidden']:
            col_type = map_tulip_type_to_spark_type(col['dataType']['type'])
            
            # Provide explicit nullability information
            nullable = True
            if 'required' in col and col['required'] is True:
                nullable = False
                
            fields.append(StructField(col['name'], col_type, nullable))
            
            # Debug logging for column types
            print(f"Column '{col['name']}': Tulip type '{col['dataType']['type']}' mapped to Spark type '{col_type}'")
            
    return StructType(fields)

def create_data_table(spark: SparkSession, table_metadata: Dict[str, Any]):
    """Create or update Delta table based on the Tulip table metadata"""
    table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{sanitize_table_name(table_metadata['label'])}"
    
    # Use explicit schema instead of string-based schema
    schema = build_table_schema(table_metadata)
    
    # Check if table already exists
    try:
        existing_table = spark.table(table_name)
        print(f"Table {table_name} already exists, checking schema compatibility")
        
        # Create a sample DataFrame to check schema compatibility
        empty_df = spark.createDataFrame([], schema)
        
        # Check if schema is compatible
        schema_compatible = validate_schema_compatibility(spark, empty_df, table_name)
        if not schema_compatible:
            print(f"Schema has changed for table {table_name}. Schema update may be required.")
            # Note: In a production setting, you would implement schema evolution strategies here
            # For this script, we'll continue with the existing table
    except:
        print(f"Creating new table {table_name}")
        # Create empty DataFrame with schema
        empty_df = spark.createDataFrame([], schema)
        # Create table
        empty_df.write.format("delta").mode("ignore").saveAsTable(table_name)
    
    return table_name

def process_record_batch(spark: SparkSession, table_name: str, schema: StructType, records: List[Dict[str, Any]]):
    """Process a batch of records with explicit schema and type handling"""
    if not records:
        return 0
        
    # Convert records to match schema
    processed_records = []
    for record in records:
        row = {}
        for field in schema.fields:
            if field.name in record:
                value = record[field.name]
                # Handle null values
                if value is None:
                    row[field.name] = None
                # Handle type conversions safely based on field type
                elif isinstance(field.dataType, DoubleType):
                    try:
                        row[field.name] = float(value) if value is not None else None
                    except (ValueError, TypeError):
                        row[field.name] = None
                        print(f"Warning: Could not convert value '{value}' to float for field '{field.name}'")
                elif isinstance(field.dataType, IntegerType):
                    try:
                        row[field.name] = int(value) if value is not None else None
                    except (ValueError, TypeError):
                        row[field.name] = None
                        print(f"Warning: Could not convert value '{value}' to integer for field '{field.name}'")
                elif isinstance(field.dataType, BooleanType):
                    try:
                        if isinstance(value, bool):
                            row[field.name] = value
                        elif isinstance(value, str):
                            row[field.name] = value.lower() in ('true', 't', 'yes', 'y', '1')
                        elif isinstance(value, (int, float)):
                            row[field.name] = bool(value)
                        else:
                            row[field.name] = None
                    except (ValueError, TypeError):
                        row[field.name] = None
                        print(f"Warning: Could not convert value '{value}' to boolean for field '{field.name}'")
                elif isinstance(field.dataType, TimestampType):
                    try:
                        row[field.name] = value
                    except (ValueError, TypeError):
                        row[field.name] = None
                        print(f"Warning: Could not convert value '{value}' to timestamp for field '{field.name}'")
                else:
                    # Default to string conversion for other types
                    try:
                        row[field.name] = str(value) if value is not None else None
                    except Exception:
                        row[field.name] = None
                        print(f"Warning: Could not convert value to string for field '{field.name}'")
            else:
                row[field.name] = None
        processed_records.append(row)
    
    # Create DataFrame with explicit schema
    df = spark.createDataFrame(processed_records, schema)
    
    # Validate schema compatibility
    schema_compatible = validate_schema_compatibility(spark, df, table_name)
    if not schema_compatible:
        print(f"Schema incompatibility detected. Trying to harmonize schema...")
        # Get table schema as a reference
        try:
            table_df = spark.table(table_name)
            for field in table_df.schema.fields:
                if field.name in df.columns:
                    df = df.withColumn(field.name, col(field.name).cast(field.dataType))
        except Exception as e:
            print(f"Error harmonizing schema: {str(e)}")
    
    # Write to table
    df.write.format("delta").mode("append").saveAsTable(table_name)
    
    return len(processed_records)

def fetch_table_data(api: TulipAPI, spark: SparkSession, table_metadata: Dict[str, Any], table_name: str):
    """Fetch and process table data in batches with robust error handling"""
    table_id = table_metadata["id"]
    schema = build_table_schema(table_metadata)
    
    total_records = 0
    offset = 0
    batch_counter = 0
    
    while True:
        try:
            print(f"Fetching batch {batch_counter} (offset {offset}) for table {table_name}")
            data = api.get_table_records(table_id, offset)
            
            if not data:
                print(f"No more data for table {table_name}")
                break
                
            # Process batch with explicit schema
            records_processed = process_record_batch(spark, table_name, schema, data)
            
            if records_processed == 0:
                break
                
            total_records += records_processed
            offset += BATCH_SIZE
            batch_counter += 1
            
            print(f"Processed {records_processed} records (total: {total_records})")
            
        except requests.exceptions.RequestException as e:
            print(f"API error fetching data for table {table_name}: {e}")
            break
        except Exception as e:
            print(f"Error processing batch for table {table_name}: {e}")
            break
    
    return total_records

# Section 6: Main Execution

In [0]:
def main():
    # Initialize API client
    config = TulipConfig.from_env()
    api = TulipAPI(config)
    
    try:
        # Get table metadata
        table_metadata = api.get_tables()
        
        # Create and update metadata tables
        create_metadata_tables(spark)
        update_metadata_tables(spark, table_metadata)
        
        # Process each table
        for table in table_metadata:
            try:
                # Create table in Unity Catalog
                table_name = create_data_table(spark, table)
                
                # Fetch and load data with robust schema handling
                total_records = fetch_table_data(api, spark, table, table_name)
                
                print(f"Successfully processed table: {table['label']} with {total_records} records")
                    
            except Exception as e:
                print(f"Error processing table {table['label']}: {e}")
                continue
        
        print("All tables processed successfully!")
        
    except Exception as e:
        print(f"Error in main process: {e}")
        raise

# Run the main process
if __name__ == "__main__":
    main()